Project Overview: Analyzing Holiday Sales Trends
Retail giants like Walmart have increasingly focused on expanding their online presence, with e-commerce playing a crucial role in overall revenue. By the end of 2022, Walmart’s online sales surged to $80 billion, accounting for 13% of its total revenue. A key factor influencing sales patterns is the occurrence of public holidays, such as the Super Bowl, Labor Day, Thanksgiving, and Christmas.

For this project, your goal is to build a data pipeline that analyzes supply and demand fluctuations around these holidays while performing an initial data exploration. You will work with two datasets: grocery sales and supplementary market data.

Data Sources
Grocery Sales Data (grocery_sales.csv)

index – Unique identifier for each row
Store_ID – Store location identifier
Date – Sales week
Weekly_Sales – Revenue generated for that store during the specified week
Supplementary Data (extra_data.parquet)

IsHoliday – Indicates if the sales week includes a public holiday (1 = Yes, 0 = No)
Temperature – Recorded temperature on the sales day
Fuel_Price – Fuel cost in the store’s region
CPI – Consumer Price Index at the time
Unemployment – Regional unemployment rate
MarkDown1 - MarkDown4 – Promotional markdown amounts
Dept – Department number within each store
Size – Store size classification
Type – Store category based on its size
Our task is to clean, integrate, and analyze these datasets to extract meaningful insights into sales trends, particularly around key holidays.

In [1]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    store_data = pd.read_csv(store_data)
    extra_df = pd.read_csv(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract("grocery_sales.csv", "extra_data.csv")

In [11]:
# Create the transform() function with one parameter: "raw_data"
import pandas as pd

def transform(raw_data):
    # Identify columns with missing values
    missing_columns = raw_data.columns[raw_data.isnull().any()]
    
    for column in missing_columns:
        # Check if the column is of numerical type
        if raw_data[column].dtype in ['float64', 'int64']:
            # Fill numerical columns with the mean for most cases
            mean_value = raw_data[column].mean()
            raw_data[column].fillna(mean_value, inplace=True)
        
        # Check if the column is categorical (object or category types)
        elif raw_data[column].dtype in ['object', 'category']:
            # Fill categorical columns with the mode (most frequent value)
            mode_value = raw_data[column].mode()[0]
            raw_data[column].fillna(mode_value, inplace=True)
        
        # Check if the column is of datetime type
        elif pd.api.types.is_datetime64_any_dtype(raw_data[column]):
            # Fill datetime columns using forward fill
            raw_data[column].fillna(method="ffill", inplace=True)
    
    # Remove rows where Weekly_Sales is not greater than 10,000
    raw_data = raw_data[raw_data["Weekly_Sales"] > 10000]
    
    # Add a "Month" column extracted from the "Date" column
    raw_data["Month"] = pd.to_datetime(raw_data["Date"]).dt.month
    
    # Drop unnecessary columns (you can adjust this as per your specific needs)
    columns_to_drop = ['level_0_x', 'level_0_y', 'index', 'Type', 'Size','Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Date']
    raw_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
   # raw_data.rename(columns={"Weekly_Sales": "Avg_Sales"}, inplace=True)
    # Return the cleaned DataFrame
    clean_data = raw_data
    return clean_data

In [12]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)

C:\Users\nosha\AppData\Local\Temp\ipykernel_9680\2161696618.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Month"] = pd.to_datetime(raw_data["Date"]).dt.month
C:\Users\nosha\AppData\Local\Temp\ipykernel_9680\2161696618.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data.drop(columns=columns_to_drop, inplace=True)


In [5]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    agg_data=clean_data.groupby(["Month"])["Weekly_Sales"].agg("mean").reset_index().round(2)
    return agg_data

In [6]:
agg_data=avg_weekly_sales_per_month(clean_data)

In [7]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
def load(cleaned_data, agg_data, cleaned_path="clean_data.csv", agg_path="agg_data.csv"):
    # Save the cleaned data to a CSV file without the index
    cleaned_data.to_csv(cleaned_path, index=False)
    
    # Save the aggregated data to a CSV file without the index
    agg_data.to_csv(agg_path, index=False)
    return cleaned_path,agg_path

In [8]:
cleaned_path,agg_path=load(clean_data, agg_data, cleaned_path="clean_data.csv", agg_path="agg_data.csv")

In [9]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(cleaned_path, agg_path):
    # Check if the cleaned data CSV file exists
    if os.path.exists(cleaned_path):
        print(f"{cleaned_path} exists.")
    else:
        print(f"{cleaned_path} does not exist.")
    
    # Check if the aggregated data CSV file exists
    if os.path.exists(agg_path):
        print(f"{agg_path} exists.")
    else:
        print(f"{agg_path} does not exist.")

In [10]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation(cleaned_path="clean_data.csv", agg_path="agg_data.csv")

clean_data.csv exists.
agg_data.csv exists.
